# 第二章 交易日历和时间计算

In [1]:
from omicron.core.timeframe import tf

print(f"{tf.day_frames[0]}~{tf.day_frames[-1]}")

print(f"{tf.week_frames[0]}~{tf.week_frames[-1]}")

print(f"{tf.month_frames[0]}~{tf.month_frames[-1]}")

20050104~20230209
20050107~20230209
20050131~20230209


在上面的API中，我们定义了行情数据的起始和结束区间。但是，在量化交易中，我们常常需要知道某个区间包含多少条数据记录。比如，当我们要计算2020年12月5日这一天，某支股票的季线、月线和10日线时，更方便的使用方法是指定结束日期，和要获取的记录条数。至于这段时间的起点是哪一天，我们并不特别关心。

由于存在节假日休市的情况，在上述场景下，要正确地计算出起始日期就更困难了。因此，Omicron提供了一个`timeframe`模块，来帮助做时间帧方面的计算。

timeframe模块以数组的形式，提供了所有的交易日、周线收盘日和月线收盘日。

可以看出，大富翁提供的日线帧从2005年1月4日开始，到2023年2月9日结束。不过，对于还没有到来的日子，具体某一天是交易日还是休市日，都还不确定，这个数据我们会实时更新的。

周线和月线帧的最后结束日并没有对齐，当然由于这一天还非常遥远，所以这里的数据也没有对错之分。

在大富翁里，你会经常看到时间帧的概念。因为对交易数据来说，数据总是在固定的时间点进行汇总，所以不应该使用普通意义上的时间概念。比如2019年1月4日的10时35分，对于1分钟线和5分钟线是有意义的，对于其它周期则是意义的。我们把这个时间点称作5分钟（或者1分钟）的一个时间帧。

timeframe模块提供了以下主要功能:

- int2time/time2int/date2int/int2date
  
    在进程间及不同的模块间传递时间数据时常常容易发生问题，比如，你无法直接往redis缓存里存入时间数据。所以，大富翁使用整数来存储日期/时间。比如，20050104代表2005年1月4日，200501041030代表2005年1月4日10时30分钟。使用这种表示，比较节省内存，同时时间之间仍然可以比较，并没有改变它们之间的次序。
    
- shift函数及衍生的各种*_shift函数

    给定一个时间，比如2019年1月4日，如果我们需要知道4个交易日前的那一天是哪一天，这时候就需要使用shift/day_shift函数。
    
- count_*_frames 计算两个时间帧之间共有多少个时间帧。
- is_trade_day 判断某天是否是交易日
- is_open_time 判断某个时间点是否处于开盘期间
- is_opening_call_auction_time 判断某个时间点是否属于早盘集合竞价时段
- is_closing_call_auction_time 判断某个时间点是否属于尾盘集合竞价时段
- floor 根据frame_type,将给定的时间对齐到最接近的上一个frame
- ceiling 对应于floor
- frame_len 对给定的分钟级别线，求一个交易日包含多少个周期
- first_frame 不同的周期，每天开盘的第一个时间帧是不一样的。比如对分钟线，第一个时间帧是9:31，对5分钟线则是9:35分。这个函数用于获得指定日期的对应周期的第一帧
- get_frames 获取给定的起始时间和结束时间间，指定的周期对应的时间帧
- get_frames_by_count，类似于get_frames，但参数不一样
- combine_time 将指定的日期与时间结合成一个新的datetime

这些功能非常基础，也十分重要。当您开始获取数据、编写策略时，会越来越依赖于它们。

Aha! jq提供的交易数据都是起始于2005年1月4日的。因此，如果你要追忆老八股当年的盛况，还得使用其它数据源。不过，对我们短线量化而言，这个数据已足够充分了。实际上，过久的历史数据如果不能正确使用，反倒会让你得出错误结论。比如，一些股票退市了，会导致幸存者偏差（幸运的是，为了不让你们出这种错误，村里很少让股票退市！）；这期间发生的重大的制度改革（比如股权分置改革就是从2005年起的），则会对某些分析方法产生影响。当然，对于短线而言，我们只关心股价，我们认为一切因素都反映在股价里。所以只要股本变动、除权除息这些事都已正确记录的话，几乎仅凭行情数据本身，我们仍然可以分析出正确结论。